# A simple task manager built with llmio

This is a demonstration of a Task Manager built with llmio.

The Task Manager is a demo application that allows you to create, update, and delete tasks.

Note that since this is only a demonstration, the tasks are stored in memory and will be lost when the application is closed.

In [1]:
import os

import llmio
from openai import AsyncOpenAI

### Define the Task model

In [2]:
from pydantic import BaseModel
from typing import Literal


# Define task model
class Task(BaseModel):
    id: int
    name: str
    description: str
    status: Literal["todo", "doing", "done"] = "todo"

# (for demonstration purposes) Define a list of tasks
TASKS: list[BaseModel] = []

### Define the agent

llmio Agents are used to define the behaviour and capabilities of the application.
The Agent requires an instruction that informs the agent its tasks and desired behaviour.

The Agent class also requires a client to interact with the language model. Here, we an openai.AsyncOpenAI client.
By default, this will talk to the OpenAI API, but other providers are also supported.

In [3]:
agent = llmio.Agent(
    instruction="You are a task manager.",
    client=AsyncOpenAI(api_key=os.environ["OPENAI_TOKEN"]),
)

### Define the agent's capabilities with tools

Tools define the capabilities of the agents built with llmio. In this case, we will give the agent the capability to list, create, update, and delete tasks.

In [4]:
# Define a tool to list tasks
@agent.tool
def list_tasks():
    print("** Listing tasks")
    return TASKS


# Define a tool to add a task
@agent.tool
def add_task(name: str, description: str, status: Literal["todo", "doing", "done"] = "todo"):
    print(f"** Adding task '{name}' with status '{status}'")
    task_id = len(TASKS) + 1
    TASKS.append(Task(id=task_id, name=name, description=description, status=status))
    return "Created task with ID " + str(task_id)


# Define a tool to update a task
@agent.tool
def update_task(task_id: int, status: Literal["todo", "doing", "done"] | None = None, description: str | None = None):
    print(f"** Updating task {task_id}")
    for task in TASKS:
        if task.id == task_id:
            task.status = status or task.status
            task.description = description or task.description
            return "Updated task status on task " + str(task_id)
    return "Task not found"


# Define a tool to remove a task
@agent.tool
def remove_task(task_id: int):
    print(f"** Removing task {task_id}")
    for task in TASKS:
        if task.id == task_id:
            TASKS.remove(task)
            return f"OK - removed task {task_id}"
    return "Task not found"

### Interact with the agent

The `Agent.speak` method is used to interact with the agent. This method sends a message to the language model and returns the response.
If tool call requests are received, the agent will execute the appropriate tool method and return the result to the language model, continuing the iteration until only a message is returned.

The `speak` method returns the received message from the language model, and conversation history so far.
Since the agent is designed to be stateless, in order for parallel execution of messages / commands to be possible, the conversation history is used to maintain the state of the agent. We will therefore store the history and use it to continue the conversation in the later interactions.

In [5]:
# Start a conversation with the agent. This initializes a message history that we can use to keep track of the conversation.
agent_messages, history = await agent.speak("Hi!")

for message in agent_messages:
    print(f"** Bot: {message}")

** Bot: Hello! How can I assist you today?


In [6]:
agent_messages, history = await agent.speak("List my tasks", history=history)

for message in agent_messages:
    print(f"** Bot: {message}")

TASKS

** Listing tasks
** Bot: You don't have any tasks listed at the moment. Would you like to add some tasks?


[]

### Add a message handler

Instead of waiting for messages to return, we can add a message handler to process agent messages as they come in.

This can be used to call a webhook or to process the message in some other way.

In this example, we will use a message handler to print the agent's response.

In [7]:
@agent.on_message
async def handle_message(message: str) -> None:
    print(f"** Bot (message_handler): '{message}'")

### Continue the interaction, but using the message handler to print the messages

In [8]:
_, history = await agent.speak("Add a grocery task with a description to buy eggs, bacon, bread, tomatoes and orange juice", history=history)

TASKS

** Adding task 'Grocery Shopping' with status 'todo'
** Bot (message_handler): 'I've added a grocery shopping task to buy eggs, bacon, bread, tomatoes, and orange juice. If you need anything else, just let me know!'


[Task(id=1, name='Grocery Shopping', description='Buy eggs, bacon, bread, tomatoes, and orange juice.', status='todo')]

In [9]:
_, history = await agent.speak("Also add a task to clean the house", history=history)

TASKS

** Adding task 'Clean the House' with status 'todo'
** Bot (message_handler): 'I've added a task to clean the house. If there's anything else you'd like to do, feel free to ask!'


[Task(id=1, name='Grocery Shopping', description='Buy eggs, bacon, bread, tomatoes, and orange juice.', status='todo'),
 Task(id=2, name='Clean the House', description='Thoroughly clean all rooms, including dusting and vacuuming.', status='todo')]

In [10]:
_, history = await agent.speak("i also need to buy some milk, can you add that to the grocery list?", history=history)

TASKS

** Updating task 1
** Bot (message_handler): 'I've updated the grocery shopping task to include milk. If you need any further assistance, just let me know!'


[Task(id=1, name='Grocery Shopping', description='Buy eggs, bacon, bread, tomatoes, orange juice, and milk.', status='todo'),
 Task(id=2, name='Clean the House', description='Thoroughly clean all rooms, including dusting and vacuuming.', status='todo')]

In [11]:
_, history= await agent.speak("The grocery task is done", history=history)

TASKS

** Updating task 1
** Bot (message_handler): 'I've marked the grocery task as done. If there's anything else you'd like to do, just let me know!'


[Task(id=1, name='Grocery Shopping', description='Buy eggs, bacon, bread, tomatoes, orange juice, and milk.', status='done'),
 Task(id=2, name='Clean the House', description='Thoroughly clean all rooms, including dusting and vacuuming.', status='todo')]

In [12]:
_, history = await agent.speak("remove all completed tasks")

TASKS

** Listing tasks
** Removing task 1
** Bot (message_handler): 'The completed task "Grocery Shopping" has been removed. If you need further assistance, feel free to ask!'


[Task(id=2, name='Clean the House', description='Thoroughly clean all rooms, including dusting and vacuuming.', status='todo')]